In [1]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256

In [2]:
from medmnist import PneumoniaMNIST
import collections

# Load the datasets
train = PneumoniaMNIST(split='train', download=True)
val = PneumoniaMNIST(split='val', download=True)
test = PneumoniaMNIST(split='test', download=True)

# Count the number of samples per class in each split
print("Train class distribution:", collections.Counter([int(x[1]) for x in train]))
print("Val class distribution:", collections.Counter([int(x[1]) for x in val]))
print("Test class distribution:", collections.Counter([int(x[1]) for x in test]))


Train class distribution: Counter({1: 3494, 0: 1214})
Val class distribution: Counter({1: 389, 0: 135})
Test class distribution: Counter({1: 390, 0: 234})


<ipython-input-2-4c7a451e89bb>:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Train class distribution:", collections.Counter([int(x[1]) for x in train]))
<ipython-input-2-4c7a451e89bb>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Val class distribution:", collections.Counter([int(x[1]) for x in val]))
<ipython-input-2-4c7a451e89bb>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Test class distribution:", collections.Counter([int(x[1

In [3]:
!pip install torch

In [3]:
from medmnist import PneumoniaMNIST
from transformers import (
    AutoImageProcessor,
    ResNetForImageClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
from torchvision import transforms
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import collections

# 1. Load data (images are PIL)
raw_train = PneumoniaMNIST(split='train', download=True)
raw_val = PneumoniaMNIST(split='val', download=True)
raw_test = PneumoniaMNIST(split='test', download=True)

# 2. Calculate class weights (for training set)
class_counts = torch.tensor([
    collections.Counter([int(x[1]) for x in raw_train])[0],  # normal
    collections.Counter([int(x[1]) for x in raw_train])[1]   # pneumonia
])
class_weights = 1. / class_counts
class_weights = class_weights / class_weights.sum()

# 3. Data augmentation and preprocessing
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),  # PIL to tensor
])
val_test_transform = transforms.Compose([
    transforms.ToTensor(),  # PIL to tensor
])

feature_extractor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

def preprocess_train(example):
    image = train_transform(example[0])
    image = image.repeat(3, 1, 1)
    encoding = feature_extractor(images=[image], return_tensors="pt")
    label = example[1]
    if isinstance(label, list):
        label = label[0]
    return {
        'pixel_values': encoding['pixel_values'].squeeze(0),
        'label': int(label)
    }

def preprocess_val_test(example):
    image = val_test_transform(example[0])
    image = image.repeat(3, 1, 1)
    encoding = feature_extractor(images=[image], return_tensors="pt")
    label = example[1]
    if isinstance(label, list):
        label = label[0]
    return {
        'pixel_values': encoding['pixel_values'].squeeze(0),
        'label': int(label)
    }


def to_hf_dataset(raw_dataset, preprocess_fn):
    data = {'image': [x[0] for x in raw_dataset], 'label': [x[1] for x in raw_dataset]}
    hf_dataset = Dataset.from_dict(data)
    hf_dataset = hf_dataset.map(lambda x: preprocess_fn((x['image'], x['label'])), remove_columns=['image'])
    return hf_dataset

train_dataset = to_hf_dataset(raw_train, preprocess_train)
val_dataset = to_hf_dataset(raw_val, preprocess_val_test)
test_dataset = to_hf_dataset(raw_test, preprocess_val_test)


<ipython-input-3-3d4b4ad1dd75>:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  collections.Counter([int(x[1]) for x in raw_train])[0],  # normal
<ipython-input-3-3d4b4ad1dd75>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  collections.Counter([int(x[1]) for x in raw_train])[1]   # pneumonia
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be abl

Map:   0%|          | 0/4708 [00:00<?, ? examples/s]

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Map:   0%|          | 0/524 [00:00<?, ? examples/s]

Map:   0%|          | 0/624 [00:00<?, ? examples/s]

In [4]:
# 2. Load model
model = ResNetForImageClassification.from_pretrained(
    "microsoft/resnet-50",
    num_labels=2,
    ignore_mismatched_sizes=True
)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 3. Metrics: Accuracy, F1-score, ROC-AUC
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    try:
        probs = torch.softmax(torch.tensor(logits), dim=1)[:,1].numpy()
        auc = roc_auc_score(labels, probs)
    except:
        auc = 0.0
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': auc
    }


In [6]:
# 4. Custom Trainer with class weighting
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # <-- Add **kwargs
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss



In [7]:
# 5. Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir='./logs',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    do_eval=True,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    report_to="none",
    seed=42
)


In [8]:
# 6. Trainer setup
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

<ipython-input-8-ddebbb744847>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [9]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,0.613700,0.573212,0.715649,0.983871,0.627249,0.766091,0.925412
2,0.323100,0.378780,0.883588,0.960674,0.879177,0.918121,0.952147
3,0.245700,0.240872,0.912214,0.969863,0.910026,0.938992,0.973246


TrainOutput(global_step=444, training_loss=0.3975591294400327, metrics={'train_runtime': 1686.4389, 'train_samples_per_second': 8.375, 'train_steps_per_second': 0.263, 'total_flos': 2.9992883801481216e+17, 'train_loss': 0.3975591294400327, 'epoch': 3.0})

In [10]:
val_results = trainer.evaluate(val_dataset)
print(val_results)

{'eval_loss': 0.24087244272232056, 'eval_accuracy': 0.9122137404580153, 'eval_precision': 0.9698630136986301, 'eval_recall': 0.910025706940874, 'eval_f1': 0.9389920424403183, 'eval_roc_auc': 0.9732457393125774, 'eval_runtime': 54.9159, 'eval_samples_per_second': 9.542, 'eval_steps_per_second': 0.31, 'epoch': 3.0}


In [11]:
# 8. Evaluate on test set
test_results = trainer.evaluate(test_dataset)
print(test_results)


{'eval_loss': 0.3634791374206543, 'eval_accuracy': 0.8701923076923077, 'eval_precision': 0.8796068796068796, 'eval_recall': 0.9179487179487179, 'eval_f1': 0.8983688833124216, 'eval_roc_auc': 0.9407736138505369, 'eval_runtime': 66.4382, 'eval_samples_per_second': 9.392, 'eval_steps_per_second': 0.301, 'epoch': 3.0}


In [16]:
model.save_pretrained("results")


In [18]:
trainer.save_model("./my_model_directory")

In [19]:
import torch
from transformers import AutoModelForImageClassification, AutoImageProcessor
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef
import numpy as np

# 1. Load your saved model and processor

model = AutoModelForImageClassification.from_pretrained("./my_model_directory")
processor = AutoImageProcessor.from_pretrained("./my_model_directory")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 2. Prepare test data (assuming test_dataset is already prepared as before)
# If not, you can use the same preprocessing as during training:
def preprocess_test(example):
    image = val_test_transform(example[0])  # PIL to tensor
    image = image.repeat(3, 1, 1)          # To 3 channels
    encoding = processor(images=[image], return_tensors="pt")
    return {'pixel_values': encoding['pixel_values'].squeeze(0), 'label': int(example[1])}

# If you have test_dataset (from before), use that:
# test_dataset = ... (from your previous code)
# If not, prepare test_dataset as in your training code

# 3. Run inference and collect predictions and labels
preds = []
labels = []
model.eval()
with torch.no_grad():
    for example in test_dataset:
        pixel_values = torch.tensor(example['pixel_values']).unsqueeze(0).to(device)
        output = model(pixel_values)
        pred = torch.argmax(output.logits, dim=1).item()
        preds.append(pred)
        labels.append(example['label'])

# 4. Calculate metrics
f1 = f1_score(labels, preds, average='binary')
accuracy = accuracy_score(labels, preds)
mcc = matthews_corrcoef(labels, preds)

print(f"F1-score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Matthews Correlation Coefficient: {mcc:.4f}")


F1-score: 0.8984
Accuracy: 0.8702
Matthews Correlation Coefficient: 0.7202
